<a href="https://colab.research.google.com/github/ShawnLiu119/dashapp-solaractivity/blob/main/SunDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Sun Dash: Monitoring and Analyzing Solar Activity
####DS 3500: Advanced Programming with Data

In [ ]:
##Set up logistic with library and data

In [105]:
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 13.1 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=25baf8d0e53915f3cded785062b62b451521259c7b12ac185b92806eee272e82
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [202]:
!pip install dash==2.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.3 MB 6.9 MB/s 
  Attempting uninstall: dash
    Found existing installation: dash 2.6.2
    Uninstalling dash-2.6.2:
      Successfully uninstalled dash-2.6.2


In [ ]:
!pip install dash-html-components                                         

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install dash-core-components                                     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:

import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime as dt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = '/content/drive/MyDrive/Dashboard_SunDash/SN_d_tot_V2.0.csv'

#"Source: WDC-SILSO, Royal Observatory of Belgium, Brussels".
#year, month, day, decimal year, SNvalue , SNerror, Nb observations

####EDA

In [4]:
df_his = pd.read_csv(path)
df_his.head()

,1818;01;01;1818.001; -1; -1.0; 0;1
0,1818;01;02;1818.004; -1; -1.0; 0;1
1,1818;01;03;1818.007; -1; -1.0; 0;1
2,1818;01;04;1818.010; -1; -1.0; 0;1
3,1818;01;05;1818.012; -1; -1.0; 0;1
4,1818;01;06;1818.015; -1; -1.0; 0;1


In [5]:
df_his = pd.read_csv(path, sep=';', names = ['year', 'month','day', 'decimal_year', 'SNvalue' ,'SNError', 'Nb_observations','obs' ])
df_his.head()

,year,month,day,decimal_year,SNvalue,SNError,Nb_observations,obs
0,1818,1,1,1818.001,-1,-1.0,0,1
1,1818,1,2,1818.004,-1,-1.0,0,1
2,1818,1,3,1818.007,-1,-1.0,0,1
3,1818,1,4,1818.010,-1,-1.0,0,1
4,1818,1,5,1818.012,-1,-1.0,0,1


In [6]:
df_dai = df_his[df_his['year']>1980]

In [7]:
#it seems that the time series data - observation starting from 1981,so truncated the data from 1981

fig1 = go.Figure(data=[go.Scatter(x=df_dai['decimal_year'], y=df_dai['SNvalue'])])
fig1.show()

dcc.Graph(figure=fig1)

Graph(figure=Figure({
    'data': [{'type': 'scatter',
              'x': array([1981.001, 1981.004, 1981.007, ..., 2022.741, 2022.744, 2022.747]),
              'y': array([218, 194, 168, ...,  86,  72,  76])}],
    'layout': {'template': '...'}
}))

In [8]:
from dash import Dash, html, dcc, Input, Output
from jupyter_dash import JupyterDash

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
#https://medium.com/@divyakarthik28/moving-average-crossover-for-stocks-using-dash-d6fc91702999

In [9]:
df_dai['year'].unique()

array([1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
       1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [21]:
app.layout = html.Div([
    html.Div([html.H1("Sunspot Observation - 1981-2022")], style={'textAlign': "center"}),
    html.Div([
        html.Div([
            html.Div([dcc.Graph(id="my-graph")], className="row", style={"margin": "auto"}),
            html.Div([html.Div(dcc.RangeSlider(id="year selection", updatemode='drag',
                                               marks={i: '{}'.format(i) for i in df_dai['year'].unique().tolist()},
                                               min=df_dai['year'].min(), max=df_dai['year'].max(), value=[1990, 2020]),
                               className="row", style={"padding-bottom": 30,"width":"100%","margin":"auto"}),
                      html.Span("Moving Average : Select Window Interval", className="row",
                                style={"padding-top": 30,"padding-left": 40,"display":"block",
                                       "align-self":"center","width":"80%","margin":"auto"}),
                      html.Div(dcc.Slider(id="select-range1", updatemode='drag',
                                          marks={i * 10: str(i * 1) for i in range(0, 31)},
                                          min=0, max=30, value=15), className="row", style={"padding": 10}),
                      # html.Div(dcc.Slider(id="select-range2", updatemode='drag',
                      #                     marks={i * 10: str(i * 10) for i in range(0, 21)},
                      #                     min=0, max=200, value=170), className="row", style={"padding": 10})
                      ], className="row")
        ], className="six columns",style={"margin-right":0,"padding":0}),
        # html.Div([
        #     dcc.Graph(id="plot-graph")
        # ], className="six columns",style={"margin-left":0,"padding":0}),
    ], className="row")
   ], className="container")


@app.callback(
    Output("my-graph", 'figure'),
    [Input("year selection", 'value'),
     Input("select-range1", 'value')])
    #  Input("select-range2", 'value')])

def update_ma(year, range1, range2):
    df_apl = df_dai[(df_dai["year"] >= year[0]) & (df_dai["year"] <= year[1])]

    rolling_mean1 = df_apl['SNvalue'].rolling(window=range1).mean()
    # rolling_mean2 =  df_ap1['SNvalue'].rolling(window=range2).mean()

    trace1 = go.Scatter(x=df_apl['day'], y=df_apl['SNvalue'],
                        mode='lines', name='SunSpot daily')
    trace_a = go.Scatter(x=df_apl['day'], y=rolling_mean1, mode='lines', yaxis='y', name=f'SMA {range1}')
    # trace_b = go.Scatter(x=df_apl['Date'], y=rolling_mean2, mode='lines', yaxis='y', name=f'SMA {range2}')

    layout1 = go.Layout({'title': 'Solar Activity Observation With Moving Average',
                         "legend": {"orientation": "h","xanchor":"left"}
                        #  "xaxis": {
                        #      "rangeselector": {
                        #          "buttons": [
                        #              {"count": 6, "label": "6M", "step": "month",
                        #               "stepmode": "backward"},
                        #              {"count": 1, "label": "1Y", "step": "year",
                        #               "stepmode": "backward"},
                        #              {"count": 1, "label": "YTD", "step": "year",
                        #               "stepmode": "todate"},
                        #              {"label": "5Y", "step": "all",
                        #               "stepmode": "backward"}
                        #          ]
                        #      }}
                         })

    figure = {'data': [trace1],
              'layout': layout1
              }
    figure['data'].append(trace_a)
    # figure['data'].append(trace_b)
    return figure


In [23]:
if __name__ == "__main__":
    app.run_server(debug='False')

Dash app running on:


<IPython.core.display.Javascript object>